In [ ]:
import pandas as pd
import numpy as np
from datetime import date
from workalendar.europe import France
import time
from dateutil import parser

import numpy as np
import matplotlib.pyplot as plt

from sklearn import ensemble
from sklearn import datasets
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error

import csv
import sys
import datetime
import os

from sklearn import ensemble
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score




import warnings
warnings.filterwarnings('ignore')


In [ ]:
df = pd.read_csv('./train_2011_2012_2013.csv',sep=";",parse_dates=True,usecols=["DATE","ASS_ASSIGNMENT","CSPL_RECEIVED_CALLS"])

In [ ]:
cal = France()

def weekend(x):
    if cal.is_working_day(x.date()):
        return 0
    else:
        return 1
def hol(x):
    if cal.is_holiday(x.date()):
        return 1
    else:
        return 0

In [ ]:
directory = os.path.join(".","sub")
cnt = 0
for root,dirs,files in os.walk(directory):
    for file in files:
        cnt += 1
        if file.endswith(".txt"):
            lines = []
            date_init = "2000-01-01"
            with open(root+'/'+file) as f:
                lines = f.readlines()
                for line in lines[1:]:

                    values = line.split("\t")
                    gg = values[0].split(' ')
                    if parser.parse(gg[0])-parser.parse(date_init) > datetime.timedelta(days=7):
                        date_init = gg[0]
                        df2 = df[df.ASS_ASSIGNMENT == values[1]]
                        mask = (df2['DATE'] <= gg[0])
                        df2 = df2.loc[mask]
                        df2['DATE'] = pd.to_datetime(df2['DATE'], coerce=True)
                        aa = df2.groupby(["DATE","ASS_ASSIGNMENT"])['CSPL_RECEIVED_CALLS'].sum()
                        df2 = aa.to_frame().reset_index()
                        df2['day'] = df2['DATE'].dt.day
                        df2['month'] = df2['DATE'].dt.month
                        df2['year'] = df2['DATE'].dt.year
                        df2['hour'] = df2['DATE'].dt.hour
                        df2['minute'] = df2['DATE'].dt.minute
                        df2['dayoff'] = df2.apply(lambda x: hol(x['DATE']), axis=1)
                        df2['weekend'] = df2.apply(lambda x: weekend(x['DATE']), axis=1)
                        df2['weekday'] = df2['DATE'].dt.dayofweek
                        df2 = df2.drop('ASS_ASSIGNMENT', 1)
                        df2 = df2.drop('DATE', 1)
                        X = df2.as_matrix()
                        A , B = X[:,1:],X[:,0]
                        params = {'n_estimators': 1, 'max_depth': 1, 'min_samples_split': 6,
                          'learning_rate': 0.5, 'loss': 'ls'}
                        clf = ensemble.GradientBoostingRegressor(**params)
                        clf.fit(A, B)
                        ytest = []
                        dtt = parser.parse(gg[0]+' '+gg[1])
                        ytest.append(dtt.day)
                        ytest.append(dtt.month)
                        ytest.append(dtt.year)
                        ytest.append(dtt.hour)
                        ytest.append(dtt.minute)
                        ytest.append(hol(dtt))
                        ytest.append(weekend(dtt))
                        ytest.append(dtt.weekday())
                        cik = clf.predict(ytest)

                        cikt = 2*int(round((float(cik))))
                        if cikt<0:
                            cikt=0
                        with open(root+'2/'+file, "a") as att_file:
                            att_file.write(values[0]+"\t"+values[1]+"\t"+str(cikt)+"\n")

                    else:
                        ytest = []
                        dtt = parser.parse(gg[0]+' '+gg[1])
                        ytest.append(dtt.day)
                        ytest.append(dtt.month)
                        ytest.append(dtt.year)
                        ytest.append(dtt.hour)
                        ytest.append(dtt.minute)
                        ytest.append(hol(dtt))
                        ytest.append(weekend(dtt))
                        ytest.append(dtt.weekday())
                        cik = clf.predict(ytest)
                        
                        cikt = 2*int(round((float(cik))))
                        if cikt<0:
                            cikt=0

                        with open(root+'2/'+file, "a") as att_file:
                            att_file.write(values[0]+"\t"+values[1]+"\t"+str(cikt)+"\n")
                            
df_tot = pd.DataFrame({})
directory = os.path.join(".","sub2")
cnt = 0
for root,dirs,files in os.walk(directory):
    for file in files:
        cnt += 1
        if file.endswith(".txt"):
            df4 = pd.read_csv(root+'/'+file,sep="\t",header=None)
            df_tot = pd.concat([df_tot,df4])

cnt = 0
with open("submission2.txt") as f:
    with open("submission3.txt", "a") as att_file:
        lines = f.readlines()
        att_file.write(lines[0])
        for line in lines[1:]:
            cnt+=1
            if cnt%1000==0:
                print cnt
            values = line.split("\t")
            att_file.write((values[0]+"\t"+values[1]+"\t"+str(df_tot[((df_tot[0] == values[0])&(df_tot[1] == values[1]))][2].get_values()[0]))+"\n")
            
with open("submission3.txt") as f:
    with open("submission.txt", "a") as att_file:
        lines = f.readlines()
        att_file.write(lines[0])
        for line in lines[1:]:
            values = line.split("\t")
            att_file.write((values[0]+"\t"+values[1]+"\t"+str(int(round((float(values[2])/2)*1.535)))+"\n"))

In [ ]:
def linExLoss(y, y_pred):
    result = 0;
    alpha = 0.1
    for yt,yp in zip(y,y_pred):
        tehta = alpha*(yt-yp)
        result += np.exp(tehta)-tehta-1
    return result/len(y)

def newScore(y, y_pred):
    return 1/linExLoss(y, y_pred)

def heldout_score(clf, X_test, y_test):
    """compute deviance scores on ``X_test`` and ``y_test``. """
    return newScore(y_test,2*clf.predict(X_test))